# DATA622 Final Project
Daniel Dittenhafer

This notebook proceeds with using the trained convolutional neural network to predict emotion for each of the face images in the provided test set.

### Import References

In [21]:
import csv
import random
import numpy as np
import emotion_model
import dwdii_transforms

random.seed(20275)

### Definitions

The model was trained on 150x150 images, therefore we will use this size of the test data as well.

In [22]:
imgDimension = []
imgDimension.append(150)
imgDimension.append(150)

### Load Kaggle Test Data

In [23]:
testDataPath = "../../facial_expressions/test"

In [24]:
testData, fileList = dwdii_transforms.load_test_data(testDataPath, imgResize=imgDimension, verboseFreq=20)

0.000000: y38.jpg
0.069930: 183a.jpg
0.139860: RamyaKrishna_71.jpg
0.209790: NA.NE2.200.jpg
0.279720: Brahmanandam_46.jpg
0.349650: 189a.jpg
0.419580: 89a.jpg
0.489510: 43a.jpg
0.559441: YM.SU2.59.jpg
0.629371: 37a.jpg
0.699301: y29.jpg
0.769231: 54b.jpg
0.839161: Brahmanandam_122.jpg
0.909091: Brahmanandam_24.jpg
0.979021: NM.SA2.99.jpg


In [25]:
testData.shape

(286, 150, 150)

In [26]:
testX = testData.reshape(testData.shape[0], 1, testData.shape[1],testData.shape[2])

### Define and Load Trained Model

In [27]:
# Map the emotions to integers for categorization later.
emotions = dwdii_transforms.emotionNumerics()
print emotions
print len(emotions)

model = emotion_model.emotion_model_jh_v5(len(emotions), verbose=True, 
                                        input_shape=(1,imgDimension[0],imgDimension[1]))

{'sadness': 6, 'neutral': 2, 'contempt': 7, 'disgust': 1, 'anger': 0, 'surprise': 4, 'fear': 5, 'happiness': 3}
8
____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
convolution2d_5 (Convolution2D)    (None, 32, 143, 143)2080        convolution2d_input_2[0][0]      
____________________________________________________________________________________________________
activation_7 (Activation)          (None, 32, 143, 143)0           convolution2d_5[0][0]            
____________________________________________________________________________________________________
maxpooling2d_5 (MaxPooling2D)      (None, 32, 71, 71)  0           activation_7[0][0]               
____________________________________________________________________________________________________
convolution2d_6 (Convolution2D)    (None, 32, 67, 67)  25632       maxpooling2

In [28]:
loadWeights = True
if loadWeights:
    #model.load_weights("dwdii-emo-150-jhv5a-Cloud.hdf5")
    model.load_weights("dwdii-emo-150-jhv5-9tf-Cloud.hdf5")

### Produce Predictions

In [29]:
predictOutput = model.predict(testX, batch_size=32, verbose=1)

286/286 [==============================] - 6s     


In [30]:
# How many predictions?
len(predictOutput)

286

In [31]:
# Show an example of the prediction output
print predictOutput[0]

[  2.88955162e-11   8.26386519e-18   9.99909699e-01   6.64880873e-10
   6.71791713e-05   7.65165112e-14   2.31399081e-05   1.72253852e-20]


In [32]:
np.array(predictOutput[0]).argmax()

2

In [33]:
numEmo = dwdii_transforms.numericEmotions()
numEmo[2]

'neutral'

In [34]:
fileList[0]

'y38.jpg'

## Persist Predictions

In this section we save the prediction results to a file in the prescribed Kaggle competition format.

In [35]:
# For each prediction output, build the row for the CSV and add to our list for saving.
outputData = []
outputData.append(["Image", "Emotion"])
for i in range(len(predictOutput)):
    arPred = np.array(predictOutput[i])
    predictionProb = arPred.max()
    predictionNdx = arPred.argmax()
    predictedEmo = numEmo[predictionNdx]
    fileName = fileList[i]
    outputData.append([fileName,predictedEmo])

In [36]:
print outputData[0:10]

[['Image', 'Emotion'], ['y38.jpg', 'neutral'], ['KA.AN2.40.jpg', 'surprise'], ['KajalAgarwal_22.jpg', 'surprise'], ['Jayamadhuri_172.jpg', 'neutral'], ['FaridaJalal_149.jpg', 'happiness'], ['NA.HA2.203.jpg', 'surprise'], ['KM.FE2.24.jpg', 'surprise'], ['JayaBhaduri_42.jpg', 'neutral'], ['FaridaJalal_315.jpg', 'happiness']]


In [37]:
with open('../output/dwdii_predictions.csv', 'w') as mycsvfile:
    dw = csv.writer(mycsvfile)
    for row in outputData:
        dw.writerow(row)

### Appendix: Sanity Check Code

In [ ]:
from scipy import misc
path = testDataPath + "/JayaBhaduri_42.jpg"
img = misc.imread(path)
img = misc.imresize(img, (150, 150))

misc.imsave("test1000.jpg", img)